<a href="https://colab.research.google.com/github/Conor-Dunlop/Disc-Golf-Form-Analysis/blob/main/disc_golf_form_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 4.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15541, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 15541 (delta 89), reused 194 (delta 48), pack-reused 15275
Receiving objects: 100% (15541/15541), 6.41 MiB | 22.09 MiB/s, done.
Resolving deltas: 100% (11203/11203), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.1 MB/s eta 0:00:

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from google.colab.patches import cv2_imshow

# import some common libraries
import numpy as np
import os, json, cv2, random
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from google.colab import files

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import tqdm
import cv2
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog
import time
import plotly.express as px

In [ ]:
cap = cv2.VideoCapture('/content/full_speed_cropped.mp4')
width  = cv2.CAP_PROP_FRAME_WIDTH
height = cv2.CAP_PROP_FRAME_HEIGHT
fps = cv2.CAP_PROP_FPS
"""while cap.isOpened():
    ret, image = cap.read()

    if not ret:
      break

    cv2_imshow(image) # Note cv2_imshow, not cv2.imshow

    cv2.waitKey(1) & 0xff

cv2.destroyAllWindows()"""
#cap.release()

'while cap.isOpened():\n    ret, image = cap.read()\n\n    if not ret:\n      break\n\n    cv2_imshow(image) # Note cv2_imshow, not cv2.imshow\n\n    cv2.waitKey(1) & 0xff\n\ncv2.destroyAllWindows()'

In [ ]:
cap.set(cv2.CAP_PROP_POS_MSEC, 150)
ret, frame = cap.read()

In [ ]:
cfg = get_cfg()   # get a fresh new config
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

[04/21 07:08:30 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...


In [ ]:
keypoints_list = []
nose_list = []
left_eye_list = []
right_eye_list = []
left_ear_list = []
right_ear_list = []
left_shoulder_list = []
right_shoulder_list = []
left_elbow_list = []
right_elbow_list = []
left_wrist_list = []
right_wrist_list = []
left_hip_list = []
right_hip_list = []
left_knee_list = []
right_knee_list = []
left_ankle_list = []
right_ankle_list = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform keypoint detection on the frame
    outputs = predictor(frame)

    # Extract keypoints from the output
    for det_keypoints in outputs["instances"].pred_keypoints:
      person_keypoint={
      "date_time":time.ctime(),
      "nose":{'x':str(det_keypoints.cpu().numpy()[0][0]),'y': str(det_keypoints.cpu().numpy()[0][1]), 'conf': str(det_keypoints.cpu().numpy()[0][2])},
      "left_eye":{'x':str(det_keypoints.cpu().numpy()[1][0]),'y': str(det_keypoints.cpu().numpy()[1][1]), 'conf': str(det_keypoints.cpu().numpy()[1][2])},
      "right_eye":{'x':str(det_keypoints.cpu().numpy()[2][0]),'y': str(det_keypoints.cpu().numpy()[2][1]), 'conf': str(det_keypoints.cpu().numpy()[2][2])},
      "left_ear":{'x':str(det_keypoints.cpu().numpy()[3][0]),'y': str(det_keypoints.cpu().numpy()[3][1]), 'conf': str(det_keypoints.cpu().numpy()[3][2])},
      "right_ear":{'x':str(det_keypoints.cpu().numpy()[4][0]),'y': str(det_keypoints.cpu().numpy()[4][1]), 'conf': str(det_keypoints.cpu().numpy()[4][2])},
      "left_shoulder":{'x':str(det_keypoints.cpu().numpy()[5][0]),'y': str(det_keypoints.cpu().numpy()[5][1]), 'conf': str(det_keypoints.cpu().numpy()[5][2])},
      "right_shoulder":{'x':str(det_keypoints.cpu().numpy()[6][0]),'y': str(det_keypoints.cpu().numpy()[6][1]), 'conf': str(det_keypoints.cpu().numpy()[6][2])},
      "left_elbow":{'x':str(det_keypoints.cpu().numpy()[7][0]),'y': str(det_keypoints.cpu().numpy()[7][1]), 'conf': str(det_keypoints.cpu().numpy()[7][2])},
      "right_elbow":{'x':str(det_keypoints.cpu().numpy()[8][0]),'y': str(det_keypoints.cpu().numpy()[8][1]), 'conf': str(det_keypoints.cpu().numpy()[8][2])},
      "left_wrist":{'x':str(det_keypoints.cpu().numpy()[9][0]),'y': str(det_keypoints.cpu().numpy()[9][1]), 'conf': str(det_keypoints.cpu().numpy()[9][2])},
      "right_wrist":{'x':str(det_keypoints.cpu().numpy()[10][0]),'y': str(det_keypoints.cpu().numpy()[10][1]), 'conf': str(det_keypoints.cpu().numpy()[10][2])},
      "left_hip":{'x':str(det_keypoints.cpu().numpy()[11][0]),'y': str(det_keypoints.cpu().numpy()[11][1]), 'conf': str(det_keypoints.cpu().numpy()[11][2])},
      "right_hip":{'x':str(det_keypoints.cpu().numpy()[12][0]),'y': str(det_keypoints.cpu().numpy()[12][1]), 'conf': str(det_keypoints.cpu().numpy()[12][2])},
      "left_knee":{'x':str(det_keypoints.cpu().numpy()[13][0]),'y': str(det_keypoints.cpu().numpy()[13][1]), 'conf': str(det_keypoints.cpu().numpy()[13][2])},
      "right_knee":{'x':str(det_keypoints.cpu().numpy()[14][0]),'y': str(det_keypoints.cpu().numpy()[14][1]), 'conf': str(det_keypoints.cpu().numpy()[14][2])},
      "left_ankle":{'x':str(det_keypoints.cpu().numpy()[15][0]),'y': str(det_keypoints.cpu().numpy()[15][1]), 'conf': str(det_keypoints.cpu().numpy()[15][2])},
      "right_ankle":{'x':str(det_keypoints.cpu().numpy()[16][0]),'y': str(det_keypoints.cpu().numpy()[16][1]), 'conf': str(det_keypoints.cpu().numpy()[16][2])}
      }
      keypoints_list.append(person_keypoint)
      nose_list.append(person_keypoint['nose'])
      left_eye_list.append(person_keypoint['left_eye'])
      right_eye_list.append(person_keypoint['right_eye'])
      left_ear_list.append(person_keypoint['left_ear'])
      right_ear_list.append(person_keypoint['right_ear'])
      left_shoulder_list.append(person_keypoint['left_shoulder'])
      right_shoulder_list.append(person_keypoint['right_shoulder'])
      left_elbow_list.append(person_keypoint['left_elbow'])
      right_elbow_list.append(person_keypoint['right_ankle'])
      left_wrist_list.append(person_keypoint['left_wrist'])
      right_wrist_list.append(person_keypoint['right_wrist'])
      left_hip_list.append(person_keypoint['left_hip'])
      right_hip_list.append(person_keypoint['right_hip'])
      left_knee_list.append(person_keypoint['left_knee'])
      right_knee_list.append(person_keypoint['right_knee'])
      left_ankle_list.append(person_keypoint['left_ankle'])
      right_ankle_list.append(person_keypoint['right_ankle'])

    # Display the frame with keypoints
    #cv2.imshow('Frame', frame)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
        #break

# Release video capture
#cap.release()
#cv2.destroyAllWindows()

In [ ]:
right_wrist_list_x = []
right_wrist_list_y = []
for i in right_wrist_list:
  right_wrist_list_x.append(i['x'])
  right_wrist_list_y.append(i['y'])

In [ ]:
print(right_wrist_list_y)

['207.82774', '208.21104', '208.2329', '209.85864', '209.3374', '210.61504', '211.29007', '210.88622', '211.54248', '214.80682', '217.33022', '224.82689', '226.71414', '242.64389', '252.44821', '253.7763', '251.3071', '244.96156', '232.6204', '217.97038', '202.17017', '162.23753', '156.22716', '163.08157', '154.75949', '151.98656', '143.48218', '147.99718', '165.92087', '183.49193', '183.00735', '183.21928', '169.11493', '125.048706', '144.05223', '127.87452', '121.964424', '126.75515', '138.9603', '203.65851', '206.97948', '269.07803', '198.4887', '193.96199', '146.45908', '165.03873', '171.67006', '172.65245', '162.45248', '171.65863', '187.23006', '183.8939', '213.40878', '217.90312', '221.51015', '220.11356', '218.65556', '220.20528', '220.52307', '220.48022', '216.25476', '224.43436', '227.69168', '233.01038', '224.36336', '204.50557', '158.17104', '208.66882', '160.25401', '203.141', '195.44241', '208.85408', '231.04733', '214.1595', '214.64433']


In [ ]:
invy = []
for i in right_wrist_list_y:
  invy.append(height-float(i))
fig = px.scatter(x=right_wrist_list_x, y=invy)
fig.update_layout(autotypenumbers='convert types')
fig.show()

In [ ]:
import math

In [ ]:
elbow_angle = []
right_wrist_list_x = []
right_wrist_list_y = []
right_shoulder_list_x = []
right_shoulder_list_y = []
right_elbow_list_x = []
right_elbow_list_y = []
for i in right_wrist_list:
  right_wrist_list_x.append(i['x'])
  right_wrist_list_y.append(float(i['y']))
for i in right_shoulder_list:
  right_shoulder_list_x.append(i['x'])
  right_shoulder_list_y.append(float(i['y']))
for i in right_elbow_list:
  right_elbow_list_x.append(i['x'])
  right_elbow_list_y.append(float(i['y']))
for i in range(len(right_wrist_list_x)):
  a = [float(right_shoulder_list_x[i])/width, float(right_shoulder_list_y[i])/height]
  b = [float(right_elbow_list_x[i])/width, float(right_elbow_list_y[i])/height]
  c = [float(right_wrist_list_x[i])/width, float(right_wrist_list_y[i])/height]
  ba = [a[0] - b[0], a[1] - b[1]]
  bc = [c[0] - b[0], c[1] - b[1]]

  angle = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
  elbow_angle.append(angle)
elbow_angle = np.asarray(elbow_angle)

In [ ]:
right_shoulder_list_x

['431.715',
 '432.51456',
 '430.18585',
 '430.3937',
 '428.9187',
 '429.5662',
 '428.1356',
 '426.60114',
 '425.63696',
 '425.0583',
 '425.78214',
 '423.19864',
 '423.6766',
 '422.2646',
 '421.0179',
 '422.60443',
 '422.8388',
 '420.9992',
 '419.62473',
 '418.67368',
 '418.01562',
 '416.98544',
 '416.5415',
 '416.27203',
 '415.00015',
 '413.7252',
 '413.39883',
 '412.5585',
 '410.39612',
 '409.76306',
 '408.86002',
 '407.61435',
 '407.84177',
 '407.50946',
 '407.1391',
 '406.27313',
 '405.99338',
 '404.8682',
 '404.61642',
 '404.01276',
 '402.47836',
 '401.94623',
 '400.85895',
 '400.24472',
 '399.3666',
 '398.7612',
 '397.16547',
 '399.50565',
 '398.2478',
 '398.58377',
 '395.0595',
 '394.4049',
 '392.41708',
 '392.5097',
 '392.11655',
 '394.06',
 '391.8756',
 '393.28613',
 '392.04608',
 '391.66577',
 '390.46207',
 '388.17902',
 '390.09094',
 '386.65054',
 '388.97546',
 '388.20337',
 '387.1051',
 '386.02423',
 '383.55948',
 '382.83676',
 '382.12308',
 '382.05667',
 '377.90793',
 '376.

In [ ]:
print(right_shoulder_list_x[0])
print(right_shoulder_list_y[0])
print(right_elbow_list_x[0])
print(right_elbow_list_y[0])
print(right_wrist_list_x[0])
print(right_wrist_list_y[0])

431.715
125.080215
478.24265
293.57947
461.934
180.28653


In [ ]:
vid_length = len(elbow_angle)
frame_times = np.arange(0, vid_length*fps, fps)
fig = px.scatter(x=frame_times, y=elbow_angle)
fig.show()

In [ ]:
vid_length = len(elbow_angle)
frame_times = np.arange(0, vid_length*fps, fps)
fig = px.scatter(x=frame_times, y=elbow_angle)
fig.update_layout(autotypenumbers='convert types')
fig.show()